# OpenCV Project_MiniPhotoshop
## 1. Only OpenCV
### Function with Trackbar and Mouse(L)
- 모자이크
- 밝기 조절
- 흑백 처리
- 영상 회전
- 사진 확대/축소
- 리셋
- 저장
- 간단한 라인 그리기

In [1]:
import os
os.getcwd()

'c:\\Users\\ChangHo Kim\\Documents\\GitHub\\SSAC_study\\Mini Project 3'

In [1]:
import cv2
import numpy as np
import math

name = input('사진을 고르시오.')
img = cv2.imread(f'data/{name}.jpg')
img = cv2.resize(img, (512, 512))

rows, cols = img.shape[:2]
val = 10
dst = np.full(img.shape, 255, dtype=np.uint8)

# 밝기 조절
def on_bright(pos):
    global img, val, dst

    array = np.full(img.shape, (val, val, val), dtype=np.uint8)
    dst = cv2.add(img, array*pos)
    cv2.imshow('image', dst)

# 흑백 처리
def gray_scale(pos):
    global img, dst
    # if pos == 1:
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('image', dst)
    
    if pos == 0:
        cv2.imshow('image', img)

# 모자이크 기능
def mosaic(pos):
    global img, dst
    rate = 15 # 모자이크에 사용할 축소 비율 (1/rate)
    if pos == 1:
        x,y,w,h = cv2.selectROI('image', img, False) # 관심영역 선택
        if w and h:
            roi = img[y:y+h, x:x+w]   # 관심영역 지정
            roi = cv2.resize(roi, (w//rate, h//rate)) # 1/rate 비율로 축소
            # 원래 크기로 확대
            roi = cv2.resize(roi, (w,h), interpolation=cv2.INTER_AREA)  
            img[y:y+h, x:x+w] = roi   # 원본 이미지에 적용
            cv2.imshow('image', img)

# 확대/축소 (보완 필요)
def img_bigger(pos):
    global img, dst

    pos = pos*1.2
    dst = cv2.resize(img, None, None, pos, pos, cv2.INTER_CUBIC)
    cv2.imshow('image', dst)

def img_smaller(pos):
    global img, dst
    if pos == 1:
        pos = pos*0.7
        dst = cv2.resize(img, None, None, pos, pos, cv2.INTER_CUBIC)
        cv2.imshow('image', dst)

    else:
        pos = (1/pos)
        dst = cv2.resize(img, None, None, pos, pos, cv2.INTER_CUBIC)
        cv2.imshow('image', dst)

# reset 기능
def reset(pos):
    global dst
    if pos == 1:
        dst = cv2.imread(f'data/{name}.jpg')
        dst = cv2.resize(dst, (512, 512))
        cv2.imshow('image', dst)

# save 기능
def save():
    global dst
    new_img = input('SAVE AS...')
    cv2.imwrite(f'data/new_img_{name}.jpg', dst)

def rotate(pos):
    global img, dst
    angle = 0
    if pos == 0:
        dst = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', dst)
    
    else:
        pos += 1
        angle += 90.0
        m = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1.0)

        dst = cv2.warpAffine(img, m, (rows, cols), borderMode=cv2.BORDER_CONSTANT,
                            borderValue=(255, 255, 255))
        cv2.imshow('image', dst)

### drawing 기능 구현 ###
green = (0, 255, 0)
red = (0, 0, 255)
mode = True
drawing = False
ix, iy = 0, 0

# line 그리기
def draw_line(event, x, y, flags, param):
    global img, ix, iy, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            if mode == True:
                cv2.line(img, (ix, iy), (x, y), red, 2, cv2.LINE_AA)
                cv2.imshow('image', img)
                ix, iy = x, y # 그림을 그리고 또 좌표 저장
                dst = img
            elif mode == False:
                cv2.line(img, (ix, iy), (x, y), green, 2, cv2.LINE_AA)
                cv2.imshow('image', img)
                ix, iy = x, y # 그림을 그리고 또 좌표 저장
                dst = img

# 창이 생성된 이후에 호출
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_line)
cv2.createTrackbar('big', 'image', 0, 3, img_bigger)
cv2.createTrackbar('samll', 'image', 0, 3, img_smaller)
cv2.createTrackbar('bright', 'image', 0, 10, on_bright)
cv2.createTrackbar('gray', 'image', 0, 1, gray_scale)
cv2.createTrackbar('mosaic', 'image', 0, 1, mosaic)
cv2.createTrackbar('reset', 'image', 0, 1, reset)
cv2.createTrackbar('rotate', 'image', 0, 3, rotate)

# result
while True:
    cv2.imshow('image', img)
    key = cv2.waitKey(0)
    if key == ord('m'):
        mode = not mode 

    elif key == 27 : # esc
        break
cv2.destroyAllWindows()

# 저장
if __name__ == '__main__':
    save()

## 2. OpenCV with GUI (Tkinter)
### Function with Button and Mouse(L)
- 밝기 조절 (밝게, 어둡게)
- 흑백 처리
- 영상 회전
- 모자이크
- 리셋
- 저장

In [2]:
import cv2
import numpy as np
from tkinter import *
from PIL import ImageTk
from PIL import Image
from tkinter import filedialog

val = 50

class App():
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.angle = 0

        # image
        self.path = filedialog.askopenfilename()
        if len(self.path) > 0:
            self.img = cv2.imread(self.path)
            self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
            self.img = np.array(self.img, dtype=np.uint8)
            self.img = cv2.resize(self.img, (512, 512))

        self.imgtk = ImageTk.PhotoImage(image=Image.fromarray(self.img))
        self.label = Label(image=self.imgtk)
        self.label.pack(side="top")

        ## 버튼 생성
        self.rotate_button = Button(self.window, text="ROTATE", width=30, command=self.rotate)
        self.rotate_button.pack()
        
        self.blur_button = Button(self.window, text="BLUR", width=30, command=self.blur_image)
        self.blur_button.pack()

        self.bright_button = Button(self.window, text="BRIGHT", width=30, command=self.bright)
        self.bright_button.pack()

        self.dark_button = Button(self.window, text="DARK", width=30, command=self.dark)
        self.dark_button.pack()

        self.gray_button = Button(self.window, text="GRAY", width=30, command=self.gray_scale)
        self.gray_button.pack()

        self.reset_button = Button(self.window, text="RESET", width=30, command=self.reset)
        self.reset_button.pack()

        self.save_button = Button(self.window, text="SAVE", width=30, command=self.save)
        self.save_button.pack()

        self.mosaic_button = Button(self.window, text="MOSAIC", width=30, command=self.mosaic)
        self.mosaic_button.pack()

        self.window.mainloop()

    ## 함수 구현
    # 모자이크 기능
    def mosaic(self):
        self.rate = 15 # 모자이크에 사용할 축소 비율 (1/rate)
        self.x, self.y, self.w, self.h = cv2.selectROI('image', self.img, False) # 관심영역 선택
        if self.w and self.h:
            self.roi = self.img[self.y:self.y+self.h, self.x:self.x+self.w]   # 관심영역 지정
            self.roi = cv2.resize(self.roi, (self.w//self.rate, self.h//self.rate)) # 1/rate 비율로 축소
            # 원래 크기로 확대
            self.roi = cv2.resize(self.roi, (self.w, self.h), interpolation=cv2.INTER_AREA)  
            self.img[self.y:self.y+self.h, self.x:self.x+self.w] = self.roi   # 원본 이미지에 적용
            self.imgtk = ImageTk.PhotoImage(image=Image.fromarray(self.img))
            
            self.label.config(image=self.imgtk)
            self.label.image = self.imgtk
    
    # 회전 기능
    def rotate(self):
        self.rows, self.cols = self.img.shape[:2]
        self.angle += 90.0
        m = cv2.getRotationMatrix2D((self.cols/2, self.rows/2), self.angle, 1.0)

        self.img = cv2.warpAffine(self.img, m, (self.rows, self.cols), borderMode=cv2.BORDER_CONSTANT,
                            borderValue=(255, 255, 255))
        self.imgtk = ImageTk.PhotoImage(image=Image.fromarray(self.img))
        
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # 블러 기능
    def blur_image(self):
        self.img = cv2.blur(self.img, (3, 3))
        self.imgtk = ImageTk.PhotoImage(image=Image.fromarray(self.img))
        
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # 밝게
    def bright(self):
        global val
        self.array = np.full(self.img.shape, (val, val, val), dtype=np.uint8)
        self.img = cv2.add(self.img, self.array)
        self.imgtk = ImageTk.PhotoImage(image= Image.fromarray(self.img))
        
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # 어둡게
    def dark(self):
        global val
        self.array = np.full(self.img.shape, (val, val, val), dtype=np.uint8)
        self.img = cv2.subtract(self.img, self.array)
        self.imgtk = ImageTk.PhotoImage(image= Image.fromarray(self.img))
        
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # 흑백 처리
    def gray_scale(self):
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        self.imgtk = ImageTk.PhotoImage(image= Image.fromarray(self.img))
        
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # reset 기능
    def reset(self):
        if len(self.path) > 0:
            self.img = cv2.imread(self.path)
            self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
            self.img = np.array(self.img, dtype=np.uint8)
            self.img = cv2.resize(self.img, (512, 512))

        self.imgtk = ImageTk.PhotoImage(image=Image.fromarray(self.img))
        self.label.config(image=self.imgtk)
        self.label.image = self.imgtk

    # save 기능
    def save(self):
        name = input('SAVE AS...')
        self.img = np.array(self.img, dtype=np.uint8)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        
        cv2.imwrite(f'data/new_img_{name}.jpg', self.img)

root = Tk()
app = App(root, "mini-photoshop")